In [1]:
# %load init.ipy
%reload_ext autoreload
%autoreload 2

from importlib import reload

import os
import sys
import logging
import warnings
import glob
# from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# import scipy as sp
# import scipy.interpolate
# import astropy as ap
import h5py
import tqdm


# --- LZK Libs ---

import zcode.math as zmath
import zcode.plot as zplot
# import zcode.astro as zastro
from zcode.constants import *
import zcode.inout as zio

import kalepy as kale


# --- Arepo / Illustris ---

import illpy
import illpy_lib
import illpy_lib.illcosmo
import illpy_lib.illbh
import illpy_lib.illbh.mergers
import illpy_lib.illbh.details
import illpy_lib.illbh.snapshots
# import illpy_lib.illbh.groupcats

from illpy_lib.illbh import BH_TYPE, KEYS


# --- Generic Setup ---

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore');
%precision %.2g

warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 12})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})


# --- Project Setup ---

# PATH_BASE = "/blue/lblecha/lkelley/arepo_sims/cosmo_zooms/dynamics-comparison"
# PATH_BASE = "/Volumes/Smally/arepo_sims/"

# SIM_NAME = "bh_new_centering"

# sim_path = os.path.join(PATH_BASE, SIM_NAME, 'output', '')

# PATH = (
# #     "/Volumes/Smally/arepo_sims/"
# #     "bh_new_centering/output"
#     "new-log_pos-test/output"
# #     "/Volumes/Smally/log_test_data/"
# )

# SIM_PATHS = [
#     "bh_new_centering",
#     "repos-potmin",
#     "unalt-dyn"
# ]

print("done")

done


In [2]:
SIM_PATH = (
#     "/net/hernquistfs3/hernquistfs3/share_root/IllustrisTNG/Runs/"
#     "L75n1820TNG/"
    "/n/holylfs/LABS/hernquist_lab/IllustrisTNG/Runs/"
    "L75n1820TNG/"
)
PROC_PATH = (
    "/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/"
    "L75n1820TNG/"
)
NUM_SNAPS = 100

In [3]:
cosmo = illpy_lib.illcosmo.Simulation_Cosmology(SIM_PATH)
# snap_scales_fname = "snap-scales.npz"
# snap_scales_fname = os.path.join(PROC_PATH, snap_scales_fname)
# if not os.path.exists(snap_scales_fname):
#     snap_scales = np.zeros(NUM_SNAPS)
#     for sn in tqdm.trange(NUM_SNAPS):
        

Found parameters file: '/n/holylfs/LABS/hernquist_lab/IllustrisTNG/Runs/L75n1820TNG/arepo/param.txt-usedvalues'
Found 100 snapshot directories
Snap 0: loading header info from '/n/holylfs/LABS/hernquist_lab/IllustrisTNG/Runs/L75n1820TNG/output/snapdir_000/snap_000.131.hdf5'


In [4]:
import illpy_lib.illbh.groupcats
from illpy_lib.illbh.groupcats import GCAT_KEYS

# gcat_s10 = illpy_lib.illbh.groupcats.Groupcats_Snap(
#     10, SIM_PATH, PROC_PATH, recreate=False, verbose=True, load=True,  # , cosmo=cosmo
# )

gcats = illpy_lib.illbh.groupcats.Groupcats(SIM_PATH, PROC_PATH, recreate=False, verbose=True, load=True)

`_load()`: recreate: False, exists: True (/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-groupcats.hdf5)
Loaded    2728779 entries from '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-groupcats.hdf5', created '2021-01-12 17:09:25.118771'


In [5]:
print(", ".join(gcats.keys()))
print("halo    = ", zmath.frac_str(gcats['halo'] >= 0))
print("subhalo = ", zmath.frac_str(gcats['subhalo'] >= 0))

ParticleIDs, SubhaloBHMass, SubhaloBHMdot, SubhaloBfldDisk, SubhaloBfldHalo, SubhaloCM, SubhaloGasMetalFractions, SubhaloGasMetalFractionsHalfRad, SubhaloGasMetalFractionsMaxRad, SubhaloGasMetalFractionsSfr, SubhaloGasMetalFractionsSfrWeighted, SubhaloGasMetallicity, SubhaloGasMetallicityHalfRad, SubhaloGasMetallicityMaxRad, SubhaloGasMetallicitySfr, SubhaloGasMetallicitySfrWeighted, SubhaloGrNr, SubhaloHalfmassRad, SubhaloHalfmassRadType, SubhaloIDMostbound, SubhaloLen, SubhaloLenType, SubhaloMass, SubhaloMassInHalfRad, SubhaloMassInHalfRadType, SubhaloMassInMaxRad, SubhaloMassInMaxRadType, SubhaloMassInRad, SubhaloMassInRadType, SubhaloMassType, SubhaloParent, SubhaloPos, SubhaloSFR, SubhaloSFRinHalfRad, SubhaloSFRinMaxRad, SubhaloSFRinRad, SubhaloSpin, SubhaloStarMetalFractions, SubhaloStarMetalFractionsHalfRad, SubhaloStarMetalFractionsMaxRad, SubhaloStarMetallicity, SubhaloStarMetallicityHalfRad, SubhaloStarMetallicityMaxRad, SubhaloStellarPhotometrics, SubhaloStellarPhotometricsM

In [6]:
import illpy_lib.illbh.snapshots
# reload(illpy_lib.illbh.snapshots)
# snap_99 = illpy_lib.illbh.snapshots.Snapshots_Snap(99, SIM_PATH, PROC_PATH, recreate=True)
snaps = illpy_lib.illbh.snapshots.Snapshots_TNG(SIM_PATH, PROC_PATH, recreate=False, verbose=False)

In [7]:
import illpy_lib.illbh.mergers
reload(illpy_lib.illbh.mergers)
mergers = illpy_lib.illbh.mergers.Mergers_TNG(SIM_PATH, PROC_PATH, recreate=False, verbose=True)

`_load()`: recreate: False, exists: True (/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-mergers.hdf5)
Loaded      30972 entries from '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-mergers.hdf5', created '2020-11-17 00:12:58.323026'


In [8]:
mergers.keys()

['BH_Mass',
 'Masses',
 'ParticleIDs',
 'scale',
 'task',
 'unique_counts',
 'unique_ids',
 'unique_indices']

In [10]:
reload(illpy_lib.illbh)
import illpy_lib.illbh.details
reload(illpy_lib.illbh.details)
# dets_t0 = illpy_lib.illbh.details.Details_TNG_Task(0, SIM_PATH, PROC_PATH, recreate=False, verbose=True, cosmo=cosmo)
# dets = illpy_lib.illbh.details.Details_TNG(SIM_PATH, PROC_PATH, recreate=False, verbose=False)
# dets_s10 = illpy_lib.illbh.details.Details_TNG_Snap(10, SIM_PATH, PROC_PATH, recreate=False, verbose=True, cosmo=cosmo)


<module 'illpy_lib.illbh.details' from '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/research/arepo/illpy_lib/illpy_lib/illbh/details.py'>

## Examine Mergers

In [11]:
def build_tree(mrgs, start=None):
    """
    The first BH is the "out" (remaining) BH, second BH is "in" (removed) BH
    """
    tree = dict()
    if start is None:
        start = 0
        
    def mstr(mm):
        msg = "{:6d} a={:.8f} task={:3d} ".format(mm, mrgs[KEYS.SCALE][mm], mrgs[KEYS.TASK][mm])
        ids = mrgs[KEYS.ID][mm]
        mass = mrgs[KEYS.MASS][mm]
        bh_mass = mrgs[KEYS.BH_MASS][mm]
        for name, tt in zip([' in', 'out'], [BH_TYPE.IN, BH_TYPE.OUT]):
            try:
                msg += "{} id={:14d} bh_mass={:.8f} mass={:.8f} ".format(
                    name, ids[tt], bh_mass[tt], mass[tt])
            except:
                print("ids  = ", ids[tt], type(ids[tt]))
                print("mass = ", mass[tt], type(mass[tt]))
                raise

        return msg

    # -- Build Merger Tree

    num = mrgs[KEYS.SCALE].size
    next = np.ones(num, np.int32) * -1
    prev = np.ones((num, 2), np.int32) * -1
    snap = np.ones(num, np.int32) * -1
    use_bh_mass = True
    sn = 0
    for mm in tqdm.tqdm(range(start, num), 'mergers'):
        mids = mrgs[KEYS.ID][mm]
        ma = mrgs[KEYS.SCALE][mm]
        while ma > cosmo.scale[sn]:
            sn += 1
            if sn == cosmo.scale.size:
                raise ValueError(f"Could not find correct snapshot for merger sca {ma:.10f}!")

        snap[mm] = sn
        if use_bh_mass:
            mass = mrgs[KEYS.BH_MASS][mm]
            if np.all(mass == 0.0):
                mass = None
                logging.warning("WARNING: ")
        else:
            mass = mrgs[KEYS.MASS][mm]

        for nn in range(mm+1, num):
            nids = mrgs[KEYS.ID][nn]
            na = mrgs[KEYS.SCALE][nn]
            # Make sure scales are not decreasing (data is already sorted; should be impossible)
            if na < ma:
                err = f"ERROR: scale regression: {mm}={ma} ==> {nn}={na}!"
                logging.error(err)
                raise ValueError(err)

            # make sure 'in' BH is never in another merger
            if mids[BH_TYPE.IN] in nids:
                err = "ERROR: 'in' BH from merger {} found in merger {}!".format(mm, nn)
                logging.error(err)
                logging.error(mstr(mm))
                logging.error(mstr(nn))
                raise ValueError(err)

            # Check if 'out' BH is in a subsequent merger
            if mids[BH_TYPE.OUT] in nids:
                next[mm] = nn
                mass_next = mrgs[KEYS.MASS][nn]
                for jj in range(2):
                    if mids[BH_TYPE.OUT] == nids[jj]:
                        prev[nn, jj] = mm
                        # Make sure masses increased
                        if mass_next[jj] < mass[BH_TYPE.OUT]:
                            err = "ERROR: BH mass loss between merger {} and {}".format(mm, nn)
                            logging.error(err)
                            logging.error(mstr(mm))
                            logging.error(mstr(nn))
                            if use_bh_mass:
                                raise ValueError(err)
                            else:
                                logging.error("Using `Mass` not `BH_Mass`... continuing!")

                        break  # if

                break  # for jj

    tree[KEYS.T_NEXT] = next
    tree[KEYS.T_PREV] = prev
    tree[KEYS.SNAP] = snap

    num_next = np.ones(num, np.int32) * -1
    num_prev = np.ones((num, 2), np.int32) * -1

    def _count_prev(mm):
        for bh in range(2):
            temp = prev[mm][bh]
            if temp >= 0:
                num_prev[mm, bh] = _count_prev(temp)
            else:
                num_prev[mm, bh] = 0

        return num_prev[mm].sum() + 1

    for mm in reversed(range(num)):
        temp = next[mm]
        if temp >= 0:
            num_next[mm] = num_next[temp] + 1
            continue

        num_next[mm] = 0
        _count_prev(mm)

    tree[KEYS.T_NUM_NEXT] = num_next
    tree[KEYS.T_NUM_PREV] = num_prev

    # -- Report statistics

    num_next = (next >= 0)
    print("Mergers with subsequent mergers: {}".format(zmath.frac_str(num_next)))
    print("Mergers that are final remnants: {}".format(zmath.frac_str(~num_next)))

    mult = num_prev[num_prev >= 0]
    print("Remnant merger-multiplicity:")
    print("\tzero = {}".format(zmath.frac_str(mult == 0)))
    print("\tave  = {:.2f}".format(mult.mean()))
    print("\tmed  = {:.2f}".format(np.median(mult)))
    print("\t{}".format(zmath.stats_str(mult)))

    return tree

# RECREATE_TREE = True
RECREATE_TREE = False
tree_fname = "tree-temp.npz"
tree_fname = os.path.abspath(tree_fname)
tree_file_exists = os.path.exists(tree_fname)
if not tree_file_exists or RECREATE_TREE:
    print(f"Tree file '{tree_fname}' exists:{tree_file_exists}!")
    tree = build_tree(mergers)
    np.savez(tree_fname, **tree)
    print(f"Saved to '{tree_fname}' size {zio.get_file_size(tree_fname)}")
else:
    tree = np.load(tree_fname)
    print(f"Loaded tree from '{tree_fname}'")

Loaded tree from '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/research/arepo/illpy_lib/notebooks/tree-temp.npz'


In [12]:
def check_tree(tree, mergers):
    tnext = tree['tree_next']
    tprev = tree['tree_prev']
    tnext_num = tree['tree_num_next']
    tprev_num = tree['tree_num_prev']

    mids = mergers[KEYS.ID]
    bhin = BH_TYPE.IN
    bhot = BH_TYPE.OUT
    print(f"IN  = {bhin}")
    print(f"OUT = {bhot}")

    def mstr(mm):
        return f"{mm} - {mids[mm][0]}, {mids[mm][1]} - next: {tnext[mm]}, prev: {tprev[mm]}"

    for ii, nn in enumerate(tqdm.tqdm(tnext)):
        this_mids = mids[ii]
        this_in = this_mids[bhin]
        this_ot = this_mids[bhot]
        err = ""
        if nn > 0:
            if this_ot not in mids[nn]:
                err += f"Out BH of {ii} not in next merger {nn}!"
                raise ValueError()

            if this_ot == mids[nn][bhot]:
                if tprev[nn][bhot] != ii:
                    err += f"Prev of {nn} out is not {ii}!"

            if this_in == mids[nn][bhin]:
                if tprev[nn][bhin] != ii:
                    err += f"Prev of {nn} in  is not {ii}!"

        for jj, pp in enumerate(tprev[ii]):
            if pp < 0:
                continue
            if this_mids[jj] != mids[pp][bhot]:
                err += f"Prev of {ii} does not match this bh {jj}!"
                break
        else:
            pp = None

        if len(err) > 0:      
            print(mstr(ii))
            if pp is not None:
                print("prev = ", mstr(pp))
            print("next = ", mstr(nn))
            raise ValueError(err)
            
    return

# check_tree(tree, mergers)

# Assemble all BH Details Entries

In [13]:
def select_at_interval(xx, dx):
    """Select a subset of array elements to achieve a certain average spacing.
    """
    beg = np.min(xx)
    end = np.max(xx)
    num = (end - beg) / dx
    num = np.int(np.ceil(num))
    # Array of optimal spacings
    yy = beg + dx * np.arange(num+1)    
    # Find locations in input nearest to the optimal spacings
    sel = zmath.argnearest(xx, yy, assume_sorted=True)
    # Also include edges
    sel = np.concatenate([[0], sel, [xx.size-1]])
    # Make sure results are all unique
    sel = np.unique(sel)
    return sel

def setup_or_expand(data, dets, size):
    setup = (data is None)
    if setup:
        data = dict()
    for kk in dets.keys():
        if kk in KEYS._DERIVED:
            continue
        vv = dets[kk]
        shp = (size,) + np.shape(vv)[1:]
        temp = np.zeros(shp, dtype=vv.dtype)
        if setup:
            data[kk] = temp
        else:
            data[kk] = np.concatenate([data[kk], temp])

    return data

def built_full_bhs(snaps, mergers, verbose=False):
    NUM_SNAPS = 100
    LAST_SNAP = NUM_SNAPS - 1
    TARGET_DELTA_SCA = 1e-3

    snaps_uids = snaps[KEYS.U_IDS]
    snaps_ulocs = snaps[KEYS.U_INDICES]
    snaps_unums = snaps[KEYS.U_COUNTS]
    mids = mergers[KEYS.ID]

    num_mrgs = mergers[KEYS.SCALE].size
    mrg_uids = mergers[KEYS.U_IDS]
    # First location (merger) each unique BH participates in (can follow the tree afterwards)
    mrg_ulocs = mergers[KEYS.U_INDICES]
    mrg_unums = mergers[KEYS.U_COUNTS]

    bhs_ids = np.zeros_like(snaps_uids)
    bhs_sca_frst = np.zeros(bhs_ids.size)
    bhs_sca_last = np.zeros(bhs_ids.size)
    bhs_snap_frst = -1 * np.ones(bhs_ids.size, dtype=int)
    bhs_snap_last = -1 * np.ones(bhs_ids.size, dtype=int)
    # bhs_mrg_frst = -1 * np.ones(bhs_ids.size, dtype=int)
    bhs_bads = np.zeros(bhs_ids.size, dtype=int)

    mrg_in_dets = np.zeros_like(mergers[KEYS.ID], dtype=bool)
    mrg_in_snaps = np.zeros_like(mrg_in_dets)

    bh_count = 0

    arr_size = int(bhs_ids.size * (1.0 / TARGET_DELTA_SCA))
    print(f"Guessing array size: {arr_size:.4e}")

    bhs = None
    bhs_arr_arr_size = 10000
    bhs_arr_ids = []
    bhs_arr_loc = []
    bhs_arr_num = []
    bhs_arr_cnt = 0
    bhs_idx = 0

    bh_no_snap = []
    bh_missing_ids = []
    bh_missing_last_scales = []
    bh_skipped_ids = []
    bh_skipped_snaps = []
    bh_bad_next_id = []
    # START = 26
    START = 0 
    
    if cosmo is None:
        cosmo = illpy_lib.illcosmo.Simulation_Cosmology(SIM_PATH)

    first = True
    last_snap_scale = 0.0
    last_sids = []
    for sn in tqdm.trange(START, NUM_SNAPS, desc='snapshots'):
        # Load this snapshot data
        snap = illpy_lib.illbh.snapshots.Snapshots_Snap(
            sn, SIM_PATH, PROC_PATH, recreate=False, verbose=False
        )
        if verbose:
            print(f"\n\nSnap {sn:03d} size: {snap.size}")

        if len(snap.keys()) == 0:
            continue

        snap_scale = snap.scale[0]
        # Load details for this snapshot
        dets = illpy_lib.illbh.details.Details_TNG_Snap(
            sn, SIM_PATH, PROC_PATH, recreate=False, verbose=False, cosmo=cosmo
        )
        if verbose:
            print(f"dets size: {dets.size}")
        if first and verbose:
            print(f"snap keys:\n\t{snap.keys()}")
            print(f"dets keys:\n\t{dets.keys()}")
            first = False

        if verbose:
            print(f"snap last: {last_snap_scale:.8e}, scale: {snap.scale[0]:.8e}")

            temp = f"{cosmo.scale[sn]:.8e}"
            if sn > 0:
                temp = f"{cosmo.scale[sn-1]:.8e}, " + temp
            print(temp)

            stats = zmath.stats_str(dets.scale, format=':.8e')
            print(f"dets scales: {stats}")

        # Make sure all details times fall within this snapshot's range
        if np.any(dets.scale - last_snap_scale < -1e-4):
            err = f"Details found before previous snapshot!"
            logging.exception(err)
            raise RuntimeError(err)

        if np.any(snap_scale - dets.scale < -1e-4):
            err = f"Details found after this snapshot!"
            logging.exception(err)
            raise RuntimeError(err)

        # ---- Go through each unique details BH
        sids = snap[KEYS.ID]
        dids = dets[KEYS.U_IDS]
        # track whether each snapshot BH is found in dets
        snap_in_dets = np.zeros(sids.size, dtype=bool)
        if verbose:
            print(f"num dets: {dids.size}")
        for bhid, beg, num in zip(dids, dets[KEYS.U_INDICES], dets[KEYS.U_COUNTS]):
            if verbose:
                print("")
            msg = f"Snap:{sn} dets BHID:{bhid}"
            end = beg + num
            # Find the index number out of BHs from all snapshots, maching this one
            all_idx = np.argmax(snaps_uids == bhid)
            # If this BH is not in any snapshots, skip it
            if(snaps_uids[all_idx] != bhid):
                bh_no_snap.append([sn, bhid])
                logging.debug(f"{msg} not in any snapshot")
                continue
            # Make sure this BH wasn't already marked as bad (shouldn't happen)    
            if (bhs_bads[all_idx] != 0):
                err = (
                    f"{msg} with {all_idx=}, {bhs_snap_frst[all_idx]=}, "
                    f"{bhs_snap_last[all_idx]=} - already marked as bad!"
                )
                logging.exception(err)
                raise RuntimeError(err)

            if verbose:
                print(f"{msg} : {all_idx=}")            

            # -- Store Details
            cut = slice(beg, end)
            dsca = dets[KEYS.SCALE][cut]
            sel = select_at_interval(dsca, TARGET_DELTA_SCA)
            num_new = sel.size
            # `bhs_idx` is the first index number for this BH's details entries
            # `bhs_end` will be the last index number (non-includsive) for this BH's details entries
            bhs_end = bhs_idx + num_new

            # Expand storage arrays if needed
            if bhs_arr_cnt + 1 > len(bhs_arr_ids):
                _pad = bhs_arr_arr_size
                bhs_arr_ids = np.concatenate([bhs_arr_ids, np.zeros(_pad, dtype=bhs_ids.dtype)])
                bhs_arr_loc = np.concatenate([bhs_arr_loc, np.zeros(_pad, dtype=int)])
                bhs_arr_num = np.concatenate([bhs_arr_num, np.zeros(_pad, dtype=int)])

            bhs_arr_ids[bhs_arr_cnt] = bhid
            bhs_arr_loc[bhs_arr_cnt] = bhs_idx
            bhs_arr_num[bhs_arr_cnt] = num_new
            bhs_arr_cnt += 1

            # Make sure `bhs` is large enough to accept new values
            if (bhs is None) or (bhs_end > bhs[KEYS.SCALE].size):
                bhs = setup_or_expand(bhs, dets, arr_size)

            for kk in dets.keys():
                if kk in KEYS._DERIVED:
                    continue
                # Add this BH's entries into the global storage
                #    `cut` selects the entries for this BH
                #    `sel` selects the subset of entries matching target spacing
                bhs[kk][bhs_idx:bhs_end] = dets[kk][cut][sel]

            # Increment the counter to the starting point for the next BH
            bhs_idx += num_new

            # -- Store meta-info

            first_snap = (bhs_ids[all_idx] == 0)
            # If BH hasnt been stored yet, this is its first snapshot, store first info
            if first_snap:
                bhs_ids[all_idx] = bhid
                bhs_snap_frst[all_idx] = sn
                bhs_sca_frst[all_idx] = dets[KEYS.SCALE][beg]
            else:
                # BH was already stored, make sure it was also found in rpevious snapshot
                if bhs_snap_last[all_idx] != sn-1:
                    err = f"{msg} skipped snapshot {sn-1}!"
                    logging.debug(err)
                    #print(err)
                    #logging.warning(err)
                    bh_skipped_ids.append(bhid)
                    bh_skipped_snaps.append(sn-1)
                    #logging.exception(err)
                    #raise RuntimeError(err)

                # If this BHID was previously seen in a snapshot but not details, set first scale
                if bhs_sca_frst[all_idx] == 0.0:
                    bhs_sca_frst[all_idx] = dets[KEYS.SCALE][beg]

            # Update the latest scale-factor for this BH
            bhs_sca_last[all_idx] = dets[KEYS.SCALE][end-1]

            # -- Snapshot: Find the index in this snapshot, matching this (details) BH
            snap_idx = np.argmax(sids == bhid)
            # Determine if details BH reaches ending snapshot
            reaches_end = (sids[snap_idx] == bhid)
            if verbose:
                print(f"{msg} : {snap_idx=}, {reaches_end=}")
            if reaches_end:
                # Update the last snapshot number this BH was found in
                bhs_snap_last[all_idx] = sn
                # Update that this snapshot BH was found in details
                snap_in_dets[snap_idx] = True

            # -- Merger: find the index in mergers this BH participates in
            mrg_idx = np.argmax(mrg_uids == bhid)
            in_merger = (mrg_uids[mrg_idx] == bhid)
            if verbose:
                print(f"{msg} : {mrg_idx=}, {in_merger=}")

            next_bhid = bhid
            if in_merger:
                # bhs_mrg_frst[all_idx] = mrg_idx   !! NOT SURE WHAT THIS IS DOING !!

                # Find the first merger index this BH participates in
                #   Note: this could be well-before or well-after this snapshot
                mrg_idx = mrg_ulocs[mrg_idx]
                # determine if 'in' or 'out' BH in merger
                which = (mergers[KEYS.ID][mrg_idx, :] == bhid)
                # Store that this merger has been found in details
                mrg_in_dets[mrg_idx, which] = True

                # Follow the merger tree to find the last merger before this snapshot
                nn = tree[KEYS.T_NEXT][mrg_idx]
                if verbose:
                    print(f"[{last_snap_scale:.10f}, {snap_scale:.10f}]")
                while (nn > 0) and (mergers[KEYS.SCALE][nn] < snap_scale):
                    temp = f"\tmm:{nn} - {mergers[KEYS.SCALE][nn]:.10f}"
                    mrg_idx = nn
                    nn = tree[KEYS.T_NEXT][nn]
                    temp += f" => mm:{nn} - {mergers[KEYS.SCALE][nn]:.10f}"
                    if verbose:
                        print(temp)

                # if the merger isn't between the prev and this snapshot, ignore it
                msca = mergers[KEYS.SCALE][mrg_idx]
                if verbose:
                    print(f"scale:{snap_scale:.10f}, merger:{mrg_idx}, scale:{msca:.10f}")
                if (msca < last_snap_scale) or (msca > snap_scale):
                    mrg_idx = None
                elif (msca <= last_snap_scale) or (msca >= snap_scale):
                    err = "MERGER AT SNAPSHOT!"
                    logging.exception(err)
                    raise RuntimeError(err)

            else:
                mrg_idx = None

            # If no mergers in this snapshot
            if mrg_idx is None:
                # If BH reaches this snapshot (i.e. end of interval), all is well
                if reaches_end:
                    continue

                # If no mergers, but BH does not reach snapshot end... something wrong!
                lvl = logging.INFO
                scales = dets[KEYS.SCALE][beg:end]
                logging.log(lvl, f"\n{msg}")
                logging.log(lvl, f"snapshot bounds: [{last_snap_scale:.10f}, {snap_scale:.10f}]")
                logging.log(lvl, "dets scales = ", scales.size, zmath.minmax(scales))
                last_scale = np.max(scales)
                if bhid in bh_missing_ids:
                    raise RuntimeError(f"{msg} already in missing IDs!!")

                # Store list of BHs that go "missing"
                bh_missing_ids.append(bhid)
                bh_missing_last_scales.append(last_scale)
                loc = snaps_ulocs[all_idx]
                num = snaps_unums[all_idx]
                logging.log(lvl, bhid, snaps_uids[all_idx], loc, num)
                scales = zmath.minmax(snaps[KEYS.SCALE][loc:(loc+num)])
                logging.log(lvl, "snap scales range = ", scales)
                err = (
                    f"{msg} did not merge in this snapshot but does not reach snapshot end!"
                    f"  {last_scale=:.10f}"
                )
                # Store that this BH has a problem
                bhs_bads[all_idx] = 1
                logging.info(err)

                # If this BH is found in a later snapshot, then something really wrong!
                last_snap = snaps[KEYS.SNAP][loc+num-1]
                check_bhid = snaps[KEYS.ID][loc+num-1]
                if last_snap > sn:
                    err = f"{msg} found in snapshot ({check_bhid=}) {last_snap} > current {sn}!"
                    logging.exception(err)
                    raise RuntimeError(err)

                # logging.exception(err)
                # raise RuntimeError(err)
                continue

            # At this point, there was a merger in this snapshot

            # Get the 'out' BH ID
            next_bhid = mergers[KEYS.ID][mrg_idx, BH_TYPE.OUT]
            if verbose:
                print(f"{msg} : {next_bhid=}")

            # If this BH (lineage) changed ID numbers
            if next_bhid != bhid:
                # Make sure the next BH *does* reach the end of this snapshot
                # find the snapshot BH matching this ID
                snap_idx = np.argmax(sids == next_bhid)
                # If the next BH reaches snapshot, all is well
                if (sids[snap_idx] == next_bhid):
                    continue

                # If the new ID is not found in snapshot, problem!
                lvl = logging.INFO
                # Note that there is an issue with the original (pre-merger) BH
                bhs_bads[all_idx] = 2
                # NOTE: the next BH `next_bhid` will be marked as problematic once we reach its details entry
                logging.log(lvl, f"\n{msg}")
                logging.log(lvl, f"snapshot bounds: [{last_snap_scale:.10f}, {snap_scale:.10f}]")
                logging.log(lvl, f"{mrg_idx=}, {next_bhid=}")
                # Check if this next-bh is found in *any* snapshots
                all_idx = np.argmax(snaps_uids == next_bhid)
                if(snaps_uids[all_idx] != next_bhid):
                    logging.log(lvl, f"next_bhid not found in any snapshot!")
                    last_snap = 0
                else:
                    loc = snaps_ulocs[all_idx]
                    num = snaps_unums[all_idx]
                    scales = zmath.minmax(snaps[KEYS.SCALE][loc:(loc+num)])
                    logging.log(lvl, f"next_bhid found in scales {scales[0]:.10f}, {scales[1]:.10f}")
                    last_snap = snaps[KEYS.SNAP][loc+num-1]

                # Because next-bh is not found in this snapshot, make sure it doesn't occur in later mergers
                logging.log(lvl, "next mergers:")                
                while mrg_idx > 0:
                    next_msca = mergers[KEYS.SCALE][mrg_idx]
                    logging.log(lvl, f"\t{mrg_idx} : {next_msca}", mergers[KEYS.ID][mrg_idx])
                    mrg_idx = tree[KEYS.T_NEXT][mrg_idx]
                    logging.log(lvl, f"\tnext = {mrg_idx}")
                    if (next_msca > snap_scale) or np.isclose(next_msca, snap_scale, rtol=1e-6, atol=1e-4):
                        err = (
                            f"Found another merger {next_msca:.10f} "
                            f"after missing snapshot scale {snap_scale:.10f}!"
                        )
                        raise ValueError(err)

                # dids, dets[KEYS.U_INDICES], dets[KEYS.U_COUNTS]
                # Find details entries for this next-bh
                dets_idx = np.argmax(dids == next_bhid)
                last_scale = np.nan
                if dids[dets_idx] != next_bhid:
                    logging.log(lvl, f"{next_bhid=} not found in this snapshots details!")
                else:
                    loc = dets[KEYS.U_INDICES][dets_idx]
                    num = dets[KEYS.U_COUNTS][dets_idx]
                    scales = dets[KEYS.SCALE][loc:(loc+num)]
                    err = f"{next_bhid=} found in {scales.size} dets: {scales[0]:.10f}, {scales[-1]:.10f}"
                    logging.log(lvl, err)
                    last_scale = scales[-1]

                if next_bhid in bh_missing_ids:
                    raise RuntimeError(f"{msg} ==> {next_bhid=} already in missing IDs!!")

                # Store this BH ID as problematic (in this particular way)
                bh_bad_next_id.append(next_bhid)
                # Beause this (next) BH ID does not reach snapshot end, it will be identified
                # later in the details' BHs loop, so we dont need to add it here (i.e. to `bh_missing_ids`)

                err = f"{msg} ==> {next_bhid=}, does not reach snapshot end!"
                logging.info(err)

                # next-BH was not found in this snapshot, but IS found in a later one... big problem!
                if last_snap > sn:
                    err = f"{msg} ==> {next_bhid=} found in snapshot {last_snap} > current {sn}!"
                    logging.exception(err)
                    raise RuntimeError(err)

                continue

            # At this point, BH was in a merger, but did *not* change ID Number

            # If in the snapshot, all is well
            if reaches_end:
                continue

            # BH did *not* reach snapshot end
            lvl = logging.INFO
            bhs_bads[all_idx] = 3
            logging.log(lvl, f"\n{msg}")
            logging.log(lvl, f"snapshot bounds: [{last_snap_scale:.10f}, {snap_scale:.10f}]")
            scales = dets[KEYS.SCALE][loc:(loc+num)]
            logging.log(lvl, f"{scales.size} dets: {scales[0]:.10f}, {scales[-1]:.10f}")
            last_scale = scales[-1]         

            # Make sure BH is not found in later snapshot
            logging.log(lvl, f"{mrg_idx=}, {next_bhid=}")
            loc = snaps_ulocs[all_idx]
            num = snaps_unums[all_idx]
            scales = zmath.minmax(snaps[KEYS.SCALE][loc:(loc+num)])
            logging.log(lvl, f"found in snap scales {scales[0]:.10f}, {scales[1]:.10f}")
            last_snap = snaps[KEYS.SNAP][loc+num-1]
            if last_snap > sn:
                err = f"{msg} found in snapshot {last_snap} > current {sn}!"
                raise RuntimeError(err)

            # Make sure BH does not participate in later merger
            logging.log(lvl, "next mergers:")                
            while mrg_idx > 0:
                next_msca = mergers[KEYS.SCALE][mrg_idx]
                logging.log(lvl, f"\t{mrg_idx} : {next_msca}", mergers[KEYS.ID][mrg_idx])
                mrg_idx = tree[KEYS.T_NEXT][mrg_idx]
                logging.log(lvl, f"\tnext = {mrg_idx}")
                if (next_msca > snap_scale) or np.isclose(next_msca, snap_scale, rtol=1e-6, atol=1e-4):
                    err = (
                        f"Found another merger {next_msca:.10f} "
                        f"after missing snapshot scale {snap_scale:.10f}!"
                    )
                    raise ValueError(err)

            if bhid in bh_missing_ids:
                raise RuntimeError(f"{msg} already in missing IDs!!")
            bh_missing_ids.append(bhid)
            bh_missing_last_scales.append(last_scale)

            err = f"{msg} does not reach snapshot end; merges, but does not change ID number!"
            logging.info(err)
            #logging.exception(err)
            #raise RuntimeError(err)

            # } for bhid in details

        # Identify any snapshot BHs not found in details
        bads = ~snap_in_dets
        if np.any(bads):
            bads = np.where(bads)[0]
            err = f"{sn=} : {bads.size} snapshot BHs not in details!"
            logging.info(err)

            for bb in bads:
                bhid = sids[bb]
                msg = f"Snap:{sn} snap BHID:{bhid}"
                all_idx = np.argmax(snaps_uids == bhid)
                # BH is in *this* snapshot, make sure it's in *all* snapshots list
                if(snaps_uids[all_idx] != bhid):
                    bh_no_snap.append([sn, bhid])
                    err = f"{msg} not in all snapshots list!"
                    print(err)
                    logging.exception(err)
                    raise RuntimeError(err)

                # Make sure we're not double counting bad BHs
                if (bhs_bads[all_idx] != 0):
                    err = f"{msg} already bad! {all_idx=}, {bhs_snap_frst[all_idx]=}, {bhs_snap_last[all_idx]=}"
                    print(err)
                    logging.exception(err)
                    raise RuntimeError(err)

                first_snap = (bhs_ids[all_idx] == 0)
                # Still store the first snapshot this ID is found in, even if problematic
                if first_snap:
                    bhs_ids[all_idx] = bhid
                    bhs_snap_frst[all_idx] = sn
                    # bhs_sca_frst[all_idx] = dets[KEYS.SCALE][beg]   # No details to use to set scale
                # If BH was already stored, make sure it was found for previous snapshot
                elif bhs_snap_last[all_idx] != sn-1:
                    err = f"{msg} skipped snapshot {sn-1}!"
                    logging.warning(err)
                    bh_skipped_ids.append(bhid)
                    bh_skipped_snaps.append(sn-1)
                    logging.exception(err)
                    raise RuntimeError(err)

                bhs_snap_last[all_idx] = sn

        if sn < NUM_SNAPS - 1:
            next_scale = cosmo.scale[sn+1]
        else:
            next_scale = 0.0

        for mm, sca in enumerate(mergers[KEYS.SCALE]):
            # In merger was before last snapshot, continue
            if (sca < last_snap_scale):
                continue

            if (sca > snap_scale):
                break

            # If merger is between last and this snapshots, make sure OUT BH is gone
            if (mergers[KEYS.ID][mm, BH_TYPE.IN] in sids):
                err = f"Merger {mm} IN BH {mergers[KEYS.ID][mm, BH_TYPE.IN]} found at snapshot {sn} end!"
                logging.exception(err)
                raise RuntimeError(err)

            # Check if BH ID is in previous snapshot
            for bh in range(2):
                mrg_in_snaps[mm, bh] = (mergers[KEYS.ID][mm, bh] in last_sids)

        # print(f"{bhs_arr_cnt:.4e} {bhs_idx=:.4e}")
        last_sids = sids
        last_snap_scale = snap_scale

    meta = dict(
        ids=bhs_ids, scale_first=bhs_sca_frst, scale_lst=bhs_sca_last,
        snap_first=bhs_snap_frst, snap_last=bhs_snap_last,
        
        bads=bhs_bads, mrg_in_dets=mrg_in_dets, mrg_in_snaps=mrg_in_snaps,
        
        no_snap=bh_no_snap, missing_ids=bh_missing_ids, missing_last_scales=bh_missing_last_scales,
        skipped_ids=bh_skipped_ids, skipped_snaps=bh_skipped_snaps, bad_next=bh_bad_next_id
    )
            
    # Trim excess elements
    for kk, vv in bhs.items():
        bhs[kk] = vv[:bhs_idx]

    # NOTE: these are invalidated by the lexsort below (? I THINK ?)
    bhs_arr_ids = bhs_arr_ids[:bhs_arr_cnt]
    bhs_arr_loc = bhs_arr_loc[:bhs_arr_cnt]
    bhs_arr_num = bhs_arr_num[:bhs_arr_cnt]

    for bh, mis in enumerate(mrg_in_snaps.T):
        print(f"{BH_TYPE.from_value(bh):3s} bad : {zmath.frac_str(~mis, frac_fmt='.3e')}")

    idx = np.lexsort((bhs[KEYS.SCALE], bhs[KEYS.ID]))
    for kk, vv in bhs.items():
        if kk in KEYS._DERIVED:
            continue
        bhs[kk] = vv[idx, ...]

    u_ids, u_inds, u_counts = np.unique(bhs[KEYS.ID], return_index=True, return_counts=True)
    # num_unique = u_ids.size
    bhs[KEYS.U_IDS] = u_ids
    bhs[KEYS.U_INDICES] = u_inds
    bhs[KEYS.U_COUNTS] = u_counts

    print(f"NBH:{u_ids.size:.2e}, entries total: {u_counts.sum():.2e}"
          f", ave: {np.mean(u_counts):.2e}, med: {np.median(u_counts):.2e}")

    return bhs, meta

# RECREATE_BHS = True
RECREATE_BHS = False
bhs_fname = "bhs-temp.npz"
bhs_meta_fname = "bhs-meta-temp.npz"
bhs_fname = os.path.abspath(bhs_fname)
bhs_meta_fname = os.path.abspath(bhs_meta_fname)
bhs_file_exists = os.path.exists(bhs_fname)
if not bhs_file_exists or RECREATE_BHS:
    print(f"BHs file '{bhs_fname}' exists:{bhs_file_exists}!")
    bhs, meta = build_full_bhs(mergers)
    np.savez(bhs_fname, **bhs)
    np.savez(bhs_meta_fname, **meta)
    print(f"Saved to '{bhs_fname}' size {zio.get_file_size(bhs_fname)}")
    print(f"Saved to '{bhs_meta_fname}' size {zio.get_file_size(bhs_meta_fname)}")
else:
    bhs = np.load(bhs_fname)
    meta = np.load(bhs_meta_fname)    
    print(f"Loaded bhs from '{bhs_fname}' and '{bhs_meta_fname}'")

Loaded bhs from '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/research/arepo/illpy_lib/notebooks/bhs-temp.npz' and '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/research/arepo/illpy_lib/notebooks/bhs-meta-temp.npz'


# Examine Different Error Types in BH data

# Make new Merger Tree

In [ ]:
idx = np.all(mrg_in_snaps, axis=-1)
print(zmath.frac_str(idx))
mrg_ids = mergers[KEYS.ID][idx, :]
mrg_sca = mergers[KEYS.SCALE][idx]
verbose = False
new_mergers = {kk: mergers[kk][idx, ...] for kk in mergers.keys() if kk not in KEYS._DERIVED}
new_tree = build_tree(new_mergers)

In [ ]:
def draw_merger_tree(mm, ax=None, follow=True, **kw):
    kw.setdefault('alpha', 0.5)
    # kw.setdefault('color', '0.5')
    # kw.setdefault('s', 100)
    kw.setdefault('marker', 'o')
    
    if ax is None:
        fig, ax = plt.subplots(figsize=[16, 12])
    
    locs = mrg_locs[mm]
    nums = mrg_nums[mm]
    
    mid = new_mergers[KEYS.ID][mm, BH_TYPE.OUT]
    bh = BH_TYPE.REMNANT
    ll = locs[bh]
    nn = nums[bh]
    warnings.warn("ignored last remnant element due to `bh_locs` construction error")
    cut = slice(ll, ll+nn-1)
    rem_sca = bhs[KEYS.SCALE][cut]
    rem_mas = bhs[KEYS.BH_MASS][cut]
    ids = bhs[KEYS.ID][cut]    
    if not np.all(ids == mid):
        print(mm, "remnant")
        print(mid)
        print(ids)
        # raise 
    
    for bh in [BH_TYPE.IN, BH_TYPE.OUT]:
        ll = locs[bh]
        nn = nums[bh]
        if nn == 0:
            print(f"EMPTY!  {mm}, {bh}")
            continue
            
        cut = slice(ll-nn+1, ll+1)
        sca = bhs[KEYS.SCALE][cut]
        mas = bhs[KEYS.BH_MASS][cut]
        ids = bhs[KEYS.ID][cut]    

        mids = new_mergers[KEYS.ID][mm, bh]
        if not np.all(ids == mids):
            print(mids)
            print(ids)
            raise
            
        # hh = ax.scatter(sca, mas, **kw)
        hh, = ax.plot(sca, mas, **kw)
        temp = dict(kw)
        # temp['color'] = hh.get_fc()[0]
        temp['color'] = hh.get_color()
        dm = np.diff(mas)
        if np.any(dm < 0.0):
            raise
        ax.plot(zmath.midpoints(sca), dm, **temp)
        
        temp['marker'] = None
        hh, = ax.plot([sca[-1], rem_sca[0]], [mas[-1], rem_mas[0]], **temp)
        
    nxt = new_tree[KEYS.T_NEXT][mm]
    if (nxt > 0) and ((follow is True) or (follow > 0)):
        print(f"{mm} ==> {nxt}")
        if follow is not True:
            follow = follow - 1
            print(f"{follow=}")
        draw_merger_tree(nxt, ax=ax, follow=follow, **kw)
    else:          
        temp = dict(kw)
        temp['color'] = hh.get_color()
        ax.plot(rem_sca, rem_mas, **temp)
        ax.plot(zmath.midpoints(rem_sca), np.diff(rem_mas), **temp)
        
    return ax

# Nice mergers: [-800, -1000]
# Missing merger in BHs: [115596010125]
MRG_NUM = -1030
# MRG_NUM = 20434
ax = draw_merger_tree(MRG_NUM, follow=0)
ax.set(yscale='log') # , ylim=[1e-3, 1e-1], xlim=[0.7, 1.0])
plt.show()
    
print(new_mergers[KEYS.ID][MRG_NUM, BH_TYPE.IN])

In [ ]:
new_tree[KEYS.T_PREV][-1030]

In [ ]:
def beg_end_for_id(bhid):
    idx = np.argmax(bhs[KEYS.U_IDS] == bhid)
    if bhs[KEYS.U_IDS][idx] != bhid:
        raise ValueError("BH ID not in values!")
        
    beg = bhs[KEYS.U_INDICES][idx]
    end = bhs[KEYS.U_COUNTS][idx]
    end += beg
    return beg, end

mrg_ids = new_mergers[KEYS.ID]
mrg_sca = new_mergers[KEYS.SCALE]
num_tot = bhs[KEYS.SCALE].size

mrg_locs = -1 * np.ones((mrg_sca.size, 3), dtype=int)
mrg_nums = -1 * np.ones((mrg_sca.size, 3), dtype=int)
for mm, (ids, msca) in tqdm.tqdm(enumerate(zip(mrg_ids, mrg_sca)), total=mrg_sca.size, desc='mergers'):
    if verbose:
        print()
    for bh in [BH_TYPE.IN, BH_TYPE.OUT]:
        bhid = ids[bh]
        msg = f"{mm} : {bh} ({BH_TYPE.from_value(bh)}), {bhid} @ {msca:.8f}"
        beg, end = beg_end_for_id(bhid)
        dsca = bhs[KEYS.SCALE][beg:end]
        dmss = bhs[KEYS.BH_MASS][beg:end]
        if dsca.size < 2:
            raise ValueError(f"{msg} :: {dsca.size=}!")
            
        dm = np.diff(dmss)
        if np.any((dm < 0.0) & (~np.isclose(0.0, dm/dmss[:-1], atol=1e-3, rtol=0.0))):
            print(msg)
            print(zmath.stats_str(dm, format=':.8e'))
            bads = np.where(dm < 0.0)[0]
            print(f"{bads.size=} {bads=}")
            print(zmath.stats_str(dm[bads], format=".8e"))
            
            bb = bads[0]
            print(f"dm = {dm[bb]:.8e} :: frac={dm[bb]/dmss[bb]:.8e}")
            cut = slice(np.max([bb-1, 0]), np.min([bb+3, dm.size]))
            print(f"dm   [{cut}] = ", zmath.str_array(dm[cut], format=':.8e'))
            print(f"mass [{cut}] = ", zmath.str_array(dmss[cut], format=':.8e'))
            print(f"scale[{cut}] = ", zmath.str_array(dsca[cut], format=':.8f'))
            print(f"id   [{cut}] = ", zmath.str_array(bhs[KEYS.ID][beg:end][cut], format=':d'))
            raise ValueError(f"{msg} :: found decreasing mass!")
        
        if msca < dsca[0]:
            raise ValueError(f"{msg} :: First detail ({dsca[0]:.8f}) is after merger ({msca:.8f})!")

        if bh == BH_TYPE.IN:
            idx = dsca.size - 1
            if dsca[idx] > msca:
                raise            
            # mrg_locs[mm, bh] = beg + idx
        else:
            # Find the index for the first detail *equal-to or past* the merger time
            idx = np.argmax(msca < dsca)
            # If *past* merger time, then the last before-merger entry is the previous one (subtract one)
            if dsca[idx] > msca:
                if idx == 0:
                    raise
                # mrg_locs[mm, bh] = beg + idx - 1
                # mrg_locs[mm, BH_TYPE.REMNANT] = beg + idx
                idx = idx - 1
            # If *equal-to* merger time, this detail is the last before-merger entry (do not modify)
            elif dsca[idx] == msca:
                if idx >= dsca.size - 1:
                    raise
                # mrg_locs[mm, bh] = beg + idx
                # mrg_locs[mm, BH_TYPE.REMNANT] = beg + idx + 1
            else:
                raise

        # Convert back to global index number, and store
        loc = beg + idx
        mrg_locs[mm, bh] = loc
        # If this is the 'out' BH, we know the remnant starts at the subsequent index number
        if bh == BH_TYPE.OUT:
            mrg_locs[mm, BH_TYPE.REMNANT] = loc + 1

        if bhs[KEYS.SCALE][loc] > msca:
            raise ValueError(f"{msg} : loc:{loc}, sca:{bhs[KEYS.SCALE][loc]:.8f}")
            
        if verbose:
            lo = np.max([idx-2, 0])
            hi = np.min([idx+2, dsca.size-1])
            print(f"{msg} : {loc=}, a={bhs[KEYS.SCALE][loc]:.8f}")
            print(f"\t{lo=}, {loc-beg}, {hi=} ({dsca.size})")       
            print(f"\ta = ", zmath.str_array(dsca[lo:hi], format=':.8f'))
            print(f"\tm = ", zmath.str_array(dmss[lo:hi], format=':.8e'))
            print(f"\tdm  = ", zmath.str_array(np.diff(dmss[lo:hi]), format=':.8e'))
        
        # -- Find the previous number of entries for this BH (after previous merger) --
        prev = new_tree[KEYS.T_PREV][mm, bh]
        # If there was no previous merger, then all entries apply (`num = idx + 1`)
        if prev < 0:
            nprev = idx + 1
        # If there was a previous merger, count entries after that
        else:
            prev_msca = new_mergers[KEYS.SCALE][prev]
            # Count the entries until merger that are after (or equal to) previous merger
            #    details entries at the same time as merger occur before the merger (hence greater-*or-equal*)
            #    count inclusively the last details entry (hence `idx+1`)
            nprev = np.count_nonzero(dsca[:idx+1] >= prev_msca)
            # The entry at `idx-nprev` should be outside of the above; i.e. before the merger
            if (idx > nprev) and (dsca[idx-nprev] >= prev_msca):
                raise ValueError(f"{msg} :: Failed to find the correct number {nprev=} of pre-merger entries!")
            # The entry at `idx-nprev+1` should be *inside* of the above; i.e. after-or-at the merger
            if dsca[idx-nprev+1] < prev_msca:
                raise ValueError(f"{msg} :: Failed to find the correct number {nprev=} of pre-merger entries!")
            if np.any(bhs[KEYS.ID][loc-nprev+1:loc+1] != bhid):
                raise ValueError(f"{msg} :: One of the previous {nprev=} details has the wrong ID number!")

        mrg_nums[mm, bh] = nprev
        
        # -- Consider the remnant BH --
        if bh == BH_TYPE.IN:
            continue
        
        # -- Find the following number of entries for this BH (before subsequent merger) --
        next = new_tree[KEYS.T_NEXT][mm]
        # If there is no next merger, then all entries apply (`num = idx + 1`)
        if next < 0:
            nnext = (end - beg) - idx
        # If there was a previous merger, count entries after that
        else:
            next_msca = new_mergers[KEYS.SCALE][next]
            # Count the entries until merger that are after (or equal to) previous merger
            #    details entries at the same time as merger occur before the merger (hence greater-*or-equal*)
            #    count inclusively the last details entry (hence `idx+1`)
            nnext = np.count_nonzero(dsca[idx:] <= next_msca)
            # The entry at `idx+nnext` should be outside of the above; i.e. after the next merger
            # print(f"{next_msca=:.8f}, {idx=}, {nnext=}")
            # print(zmath.str_array(dsca[idx:idx+nnext+2], format=':.8f'))
            # print(zmath.str_array(dsca[idx:idx+nnext+2] <= next_msca, format=':10d'))
            if (loc+nnext < num_tot) and ((bhs[KEYS.SCALE][loc+nnext] <= next_msca) and (bhs[KEYS.ID][loc+nnext] == bhid)):
                raise ValueError(f"{msg} :: Failed to find the correct number {nnext=} of post-merger entries!")
            # The entry at `idx+nnext-1` should be *inside* of the above; i.e. after the next merger
            if bhs[KEYS.SCALE][loc+nnext-1] > next_msca:
                raise ValueError(f"{msg} :: Failed to find the correct number {nnext=} of post-merger entries!")
            if np.any(bhs[KEYS.ID][loc:loc+nnext] != bhid):
                raise ValueError(f"{msg} :: One of the following {nnext=} details has the wrong ID number!")

        mrg_nums[mm, BH_TYPE.REMNANT] = nnext

    loc = mrg_locs[mm, 2]
    if verbose:
        print(f"{msg} : {loc=}, a={bhs[KEYS.SCALE][loc]:.8f}")
    if (bhs[KEYS.SCALE][loc] <= msca) or (bhs[KEYS.SCALE][loc-1] > msca):
        raise ValueError()

    test_masses = [bhs[KEYS.BH_MASS][ll] for ll in mrg_locs[mm]]
    test_remn = np.sum(test_masses[:2])
    test_err = (test_masses[-1] - test_remn) / np.mean(test_masses[:2])
    if (test_err < 0.0) and not np.isclose(test_remn, test_masses[-1], rtol=1e-5, atol=0):
        print(msg)
        print(test_masses)
        print(f"{test_remn:.8f}, {test_masses[-1]:.8f} :: {test_err:.8e}")
        print()
        raise ValueError(f"{msg} :: Total mass of remnant too low!")
                  


In [ ]:
# zmath.frac_str(bhs_mrg_frst >= 0, frac_fmt='.4e')
zmath.frac_str(mrg_in_dets > 0)

In [ ]:
# bhs_ids = np.zeros_like(snaps_uids)
# bhs_sca_frst = np.zeros(bhs_ids.size)
# bhs_sca_last = np.zeros(bhs_ids.size)
# bhs_snap_frst = -1 * np.ones(bhs_ids.size, dtype=int)
# bhs_snap_last = -1 * np.ones(bhs_ids.size, dtype=int)
# bhs_mrg_frst = -1 * np.ones(bhs_ids.size, dtype=int)
# bhs_bads = np.zeros(bhs_ids.size, dtype=int)

# mrg_in_dets = np.zeros_like(mergers[KEYS.ID], dtype=bool)
# mrg_in_snaps = np.zeros_like(mrg_in_dets)

# bh_count = 0

# bhs = dict()

# bh_no_snap = []
# bh_missing_ids = []
# bh_missing_last_scales = []
# bh_skipped_ids = []
# bh_skipped_snaps = []
# bh_bad_next_id = []


print(f"Merger BHs in dets : {zmath.frac_str(mrg_in_dets > 0)}")
print(f"              both : {zmath.frac_str(np.all(mrg_in_dets > 0, axis=1))}")
print(f"Merger BHs in snaps: {zmath.frac_str(mrg_in_snaps > 0)}")
print(f"              both : {zmath.frac_str(np.all(mrg_in_snaps > 0, axis=1))}")
print(f"Tree snaps: {zmath.frac_str(tree[KEYS.SNAP] > 0)}, all: {np.all(tree[KEYS.SNAP] > 0)}")
print("\n")



print(f"BHs with details and no snapshots: {len(bh_no_snap)}")
print(f"BHs that go missing: {len(bh_missing_ids)}")
if len(bh_missing_ids) > 0:
    fig, ax = plt.subplots(figsize=[8, 4])
    ax.hist(bh_missing_last_scales, bins=80, density=True)
    ax.plot(*kale.pdf(bh_missing_last_scales))
    kale.plot.draw_carpet(bh_missing_last_scales, ax=ax)
    xx = bh_missing_last_scales
    yy = np.random.normal(-1.0, 0.5, len(xx))
#     ax.scatter(xx, yy, color='r', alpha=0.25)
    
unique_missing = np.unique(bh_missing_ids)
if unique_missing.size != len(bh_missing_ids):
    print(f"Unique missing BHs is {unique_missing.size}!")
    
print(f"BHs skipping snapshots: {len(bh_skipped_ids)}")
unique_skipped = np.unique(bh_skipped_ids)
if unique_skipped.size != len(bh_skipped_ids):
    print(f"Unique skipped BHs is {unique_skipped.size}")
        
print("\n")
        
        
bads = np.where(bhs_ids == 0)[0]
print(f"Empty `bhs_ids`: {bads.size} : {bads=}")

print(f"Bad BHs: {zmath.frac_str(bhs_bads > 0)}")
print(zmath.stats_str(bhs_bads))
for ii in range(1, 4):
    bads = np.where(bhs_bads == ii)[0]
    print(f"{ii} - {bads.size}: {bads=}")

sel = (bhs_ids > 0)

bads = (bhs_sca_frst[sel] > bhs_sca_last[sel])
if np.any(bads):
    bads = np.where(bads)[0]
    print(f"sca first after last!  {bads.size} : {bads=}")
    print(bhs_sca_frst[sel][bads], bhs_sca_last[sel][bads])
    
bads = (bhs_snap_frst[sel] > bhs_snap_last[sel])
if np.any(bads):
    bads = np.where(bads)[0]
    print(f"snap first after last!  {bads.size} : {bads=}")
    print(bhs_snap_frst[sel][bads], bhs_snap_last[sel][bads])
    
    
        
plt.show()

In [ ]:
bhs.keys()

In [ ]:
print(f"Bad Merger locations:", zmath.frac_str(mrg_locs < 0))

In [ ]:
import datetime

In [ ]:
da = 1e-4
zbeg = 5.0

abeg = cosmo._z_to_a(zbeg)
aend = 1.0
print(f"a: [{abeg:.6f}, {aend:.6f}]")

aa = aend - da * np.arange(0, (aend//da)+2)
aa = aa[::-1]
aa = aa[(aa >= abeg)]

tage = cosmo.age(cosmo._a_to_z(aa)).to('Myr').value
dt = np.diff(tage)
ta = zmath.midpoints(tage)
plt.plot(ta, dt)
plt.show()

In [ ]:
DELTA_A = 1e-4
# DELTA_A = 3e-3

def select_at_interval_one(xx, dx):
    beg = np.min(xx)
    end = np.max(xx)
    num = (end - beg) / dx
    num = np.int(np.ceil(num))
    # Array of optimal spacings
    yy = beg + dx * np.arange(num+1)    
    # Find locations in input nearest to the optimal spacings
    sel = zmath.argnearest(xx, yy, assume_sorted=True)
    # Also include edges
    sel = np.concatenate([[0], sel, [xx.size-1]])
    # Make sure results are all unique
    sel = np.unique(sel)
    return sel
    
def select_at_interval_two(xx, target):
    sel = np.zeros_like(xx, dtype=bool)
    sel[0] = True
    sel[-1] = True

    dx = np.concatenate([[0.0], np.diff(xx)])
    
    idx = 1
    step = 0.0
    while idx < xx.size - 1:
        step += dx[idx]
        if step > target:
            if sel[idx-1]:
                sel[idx] = True
                step = 0.0
            else:
                sel[idx-1] = True
                step = dx[idx]
            
        idx += 1
    
    return sel
    
abeg = np.random.uniform(0.0, 0.5)

num = int(10.0 / DELTA_A)
da = zmath.log_normal_base_10(DELTA_A, 0.5, num)
aa = np.concatenate([[abeg], da])
aa = np.cumsum(aa)
aa = np.concatenate([aa[(aa < 1.0)], [1.0]])
print(f"{aa.size=}")

target = DELTA_A * 10.0
TIMES = 1e3

functions = [select_at_interval_one, select_at_interval_two]

fig, axes = plt.subplots(figsize=[16, 6], ncols=len(functions) + 1)

for ax in axes[:-1]:
    ax.scatter(np.arange(aa.size), aa, c='0.5', alpha=0.5, s=100)

ax = axes[-1]
ax.hist(np.log10(np.diff(aa)), bins='auto', color='0.5', alpha=0.5, density=True)
ax.axvline(np.log10(target), color='b', ls='--', alpha=0.5)

for jj, func in enumerate(functions):
    beg = datetime.datetime.now()
    for ii in range(int(TIMES)):
        sel = func(aa, target)

    dur = datetime.datetime.now() - beg
    print(f"Duration over {TIMES} tests: {dur.total_seconds():.4e}")
        
    bb = aa[sel]
    *_, hh = axes[-1].hist(np.log10(np.diff(bb)), bins='auto', density=True, alpha=0.5)
    color = hh[0].get_fc()
    
    cc = np.ones_like(aa) * np.nan
    cc[sel] = aa[sel]
    hh = axes[jj].scatter(np.arange(cc.size), cc, color=color, alpha=0.5, s=100, marker='*')
    for c in cc:
        axes[jj].axhline(c, color=color)


plt.show()

In [ ]:
DELTA_A = 4e-3

def select_at_interval(xx, dx):
    beg = np.min(xx)
    end = np.max(xx)
    num = (end - beg) / dx
    num = np.int(np.ceil(num))
    yy = beg + dx * np.arange(num+1)
    print(f"xx={zmath.stats_str(xx)}")
    print(f"dx={zmath.stats_str(np.diff(xx))}")
    print(f"yy={zmath.stats_str(yy)}")
    print(f"dy={zmath.stats_str(np.diff(yy))}")
    print(f"{xx.size=}, {yy.size=}")
    
    sel = zmath.argnearest(xx, yy, assume_sorted=True)
    sel = np.concatenate([[0], sel, [xx.size-1]])
    sel = np.unique(sel)
    
    # print(f"{zmath.frac_str(sel)}")
    return sel
    
abeg = np.random.uniform(0.0, 0.5)

num = int(10.0 / DELTA_A)
da = zmath.log_normal_base_10(DELTA_A, 0.5, num)
aa = np.concatenate([[abeg], da])
aa = np.cumsum(aa)
aa = np.concatenate([aa[(aa < 1.0)], [1.0]])

target = DELTA_A * 10.0
sel = select_at_interval(aa, target)
bb = aa[sel]

fig, axes = plt.subplots(figsize=[16, 6], ncols=2)

ax = axes[0]
ax.scatter(np.arange(aa.size), aa, c='r', alpha=0.5, s=100)
cc = np.ones_like(aa) * np.nan
cc[sel] = aa[sel]
ax.scatter(np.arange(cc.size), cc, c='b', alpha=0.5, s=100, marker='*')

ax = axes[1]
ax.hist(np.log10(np.diff(aa)), bins='auto', color='r', alpha=0.5)
ax.hist(np.log10(np.diff(bb)), bins='auto', color='b', alpha=0.5)
ax.axvline(np.log10(target), color='b', ls='--', alpha=0.5)

plt.show()

In [ ]:
tt = 100000851369
bad = np.where(snaps[KEYS.U_IDS] == tt)[0][0]
print(tt, bad)
loc = snaps[KEYS.U_INDICES][bad]
num = snaps[KEYS.U_COUNTS][bad]
print(loc, num)
cut = slice(loc, loc+num)
snap_nums = snaps[KEYS.SNAP][cut]
snap_scas = snaps[KEYS.SCALE][cut]
print(snap_nums)
print(snap_scas)


In [ ]:
dets_32 = illpy_lib.illbh.details.Details_TNG_Snap(32, SIM_PATH, PROC_PATH, recreate=False, verbose=False, cosmo=cosmo)

In [ ]:
bad = np.where(dets_32[KEYS.U_IDS] == tt)[0][0]
print(tt, bad)
loc = dets_32[KEYS.U_INDICES][bad]
num = dets_32[KEYS.U_COUNTS][bad]
print(loc, num)
cut = slice(loc, loc+num)
snap_nums = dets_32[KEYS.ID][cut]
snap_scas = dets_32[KEYS.SCALE][cut]
print(snap_nums)
print(snap_scas)
